In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from datetime import datetime

#from selenium.webdriver.chrome.service import Service as ChromeService
#from webdriver_manager.chrome import ChromeDriverManager

/Users/sebastianbaquero/Code/Sport Betting Stats/wsenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
browser=webdriver.Firefox()

# get source code
browser.get("https://www.legalsportsreport.com/sports-betting/revenue")
html = browser.page_source
time.sleep(2)
#print(html)

# close web browser
browser.close()

In [3]:
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Get States Revenue Table
table = soup.find('table', attrs={'class':'tablepress tablepress-id-Rev_Markets'})
table_body = table.find('tbody')

In [5]:
fieldsStateRevenues = ["state", "handle","revenue","hold","taxes"]
monthlyRevenues = ["date", "handle","revenue","hold","taxes"]
monthlyRevenuesByState = ["state","date", "handle","revenue","hold","taxes"]


In [6]:
stateRevenueInfo = []
rows = table_body.find_all('tr')
#print(len(rows))
for row in rows:
    cols = row.find_all('td')
    rowDict = {}
    for index, col in enumerate(cols):
        if index == 0:
            rowDict['state'] = col.text.strip()
        elif index == 1:
            if col.text.strip() != '?':
                rowDict['handle'] = int(col.text.strip().replace("$","").replace(",",""))
            else:
                rowDict['handle'] = 0
        elif index == 2:
            if col.text.strip() != '?':
                rowDict['revenue'] = int(col.text.strip().replace("$","").replace(",",""))
            else:
                rowDict['revenue'] = 0
        elif index == 3:
            if col.text.strip() != '?':
                rowDict['hold'] = col.text.strip().replace("%","").replace(",",".")
            else:
                rowDict['hold'] = '0'
        elif index == 4:
            if col.text.strip() != '?':
                rowDict['taxes'] = int(col.text.strip().replace("$","").replace(",",""))
            else:
                rowDict['taxes'] = 0

    stateRevenueInfo.append(rowDict)


In [7]:
#stateRevenueInfo

In [8]:
# Get Monthly Revenue Table
table = soup.find('table', attrs={'class':'tablepress tablepress-id-Rev_Months'})
table_body = table.find('tbody')

In [9]:
def convert_to_date(date_str):
    # Parse the input string
    date_obj = datetime.strptime(date_str, "%B %Y")
    # Convert to the desired format
    formatted_date = date_obj.strftime("%Y-%m-%d")
    return formatted_date

In [10]:
monthlyRevenueInfo = []
rows = table_body.find_all('tr')
#print(len(rows))
for row in rows:
    cols = row.find_all('td')
    rowDict = {}
    for index, col in enumerate(cols):
        if index == 0:
            rowDict['date'] = convert_to_date(col.text.strip())

        elif index == 1:
            rowDict['handle'] = int(col.text.strip().replace("$","").replace(",",""))

        elif index == 2:
            rowDict['revenue'] = int(col.text.strip().replace("$","").replace(",",""))

        elif index == 3:
            rowDict['hold'] = col.text.strip().replace("%","").replace(",",".")

        elif index == 4:
            rowDict['taxes'] = int(col.text.strip().replace("$","").replace(",",""))

    monthlyRevenueInfo.append(rowDict)

In [11]:
#monthlyRevenueInfo

In [12]:
states = ["AZ","AR","CO","CT","DE","DC","IL","IN","IA","KS","KY","LA","ME","MD dataTable","MA","MI","MS","MT","NV","NH","NJ","NY","NC","OH","OR","PA","RI","SD","TN","VT","VA","WV","WY"]
noData = ["Florida", "Nebraska","New Mexico","North Dakota","Washington","Wisconsin",]

In [13]:
# Get Monthly Revenue by State
infoRevenueByStateAndMonth = []
for state in states:
    table = soup.find('table', attrs={'class':f'tablepress tablepress-id-Rev{state}'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        tempState = {}
        tempState['state'] = state
        cols = row.find_all('td')
        for index,col in enumerate(cols):
            if index == 0:
                tempState['date'] = convert_to_date(col.text.strip())
            elif index == 1:
                if col.text.strip() != '?':
                    tempState['handle'] = int(col.text.strip().replace("$","").replace(",",""))

                else:
                    tempState['handle'] = 0

            elif index == 2:
                if col.text.strip() != '?':
                    tempState['revenue'] = int(col.text.strip().replace("$","").replace(",",""))

                else:
                    tempState['revenue'] = 0

            elif index == 3:
                if col.text.strip() != '?':
                    tempState['hold'] = col.text.strip().replace("%","").replace(",",".")
                else:
                    tempState['hold'] = '0'

            elif index == 4:
                if col.text.strip() != '?':
                    tempState['taxes'] = int(col.text.strip().replace("$","").replace(",",""))

                else:
                    tempState['taxes'] = 0
                    

        infoRevenueByStateAndMonth.append(tempState)
        




In [14]:
#infoRevenueByStateAndMonth

In [15]:
# Get Each state detail
#table = soup.find('table', attrs={'class':'cta-row row-body'})

In [16]:
#print(table)

In [17]:
# Write Files
import csv
# Write revenue by state
filename = "revenue_by_state.csv"

# writing to csv file
with open(filename, 'w') as csvfile:
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames=fieldsStateRevenues)

    # writing headers (field names)
    writer.writeheader()

    # writing data rows
    writer.writerows(stateRevenueInfo)

In [18]:
# Write monthly revenue
filename = "revenue_by_month.csv"

# writing to csv file
with open(filename, 'w') as csvfile:
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames=monthlyRevenues)

    # writing headers (field names)
    writer.writeheader()

    # writing data rows
    writer.writerows(monthlyRevenueInfo)

In [19]:
# Write monthly revenue by state
filename = "revenue_by_state_by_month.csv"

# writing to csv file
with open(filename, 'w') as csvfile:
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames=monthlyRevenuesByState)

    # writing headers (field names)
    writer.writeheader()

    # writing data rows
    writer.writerows(infoRevenueByStateAndMonth)
